Anticipation of moving stimuli by retina
Nature 1999 M. Berry

Modeling the response of a ganglion cell with gain control

Update History: creation Jan 1, 2018 by CKC

Sept 20, 2018 to include spatial part
Oct  31, 2018 to have normalized coordinate systems with dx and dt



In [5]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# use of two Gussians (one inverted) to approx the receptive field
# or spatial filter
# parameters are chosen to have min at +1 and -1

def k_x(x):
    rc = 0.5
    rs = 0.65
    
    kc = 1.3
    
    ks = 0.9
    
    a = x*x/2/rc/rc
    b = x*x/2/rs/rs
    return kc*np.exp(-a) - ks*np.exp(-b)


In [7]:
# generate an array with npts,maxval,minval
def arrgen(npts,maxval,minval):
    a = np.zeros(npts)
    constant = (maxval-minval)/(npts-1)
    for i in range(npts):
        a[i]= minval + i*constant
    return(a)

In [8]:
def xtoi(x,xnpts,xrange,x_offset): # convert position x into index i
    index = ((x-x_offset)/xrange)*xnpts
    return int(np.round(index))

In [9]:
# test of k_x and arrgen ;plot receptive field y = k_x(x)
#plt.plot(k_x(x)) # or plt.plot(x,y)
#plt.show()

gg = np.zeros(xnpts)
for i in range(xnpts):
    gg[i]=k_x(x[i])
plt.plot(gg)  

NameError: name 'x' is not defined

In [ ]:
# square pulse stimulation
def s(x,t,width,speed,x0):
    val = speed*t + x0 # position of leading edge
    if x > val-width and x < val:
        return 1.0
    else:
        return 0.0

In [ ]:
# square pulse stimulation
def s_d(x,t,width,speed,x0):
    val = speed*t + x0 # position of leading edge
    if x > val-width and x < val:
        return 0.0
    else:
        return 1.0

In [ ]:
# test of a moving bar s(x,t) (=0 within the bar, =1 otherwise)

dx = x[1]-x[0]

speed = 0.1
width = 1.0
x0 = xmin + width

y = np.zeros(xnpts)
for j in range(10):
    t = (j+1)*10
    print(t)
    for i in range(xnpts):
        y[i] =  s_d(x[i],t,width,speed,x0)#dark bar
    plt.plot(x,y) # plot the shape of the pulse at time t

plt.show()

In [ ]:
# modeling biphasic response of ganglion cell
# This is an off cell
# with a = 20/16.5 b = 0.3 c = 0.008 and t in msec
# k_t will be zero after 30 msec

def k_t(a,b,c,t):
    if t>0:
        return -a*np.sin(b*t)*np.exp(-c*t*t)
    else:
        return 0.0

In [ ]:
# check biphasic response



nstep = 100 # time step
dt = 1    # 1 msec
duration = nstep*dt 

#Checking biphasic response

a = 20/16.5
b = 0.3
c = 0.008

ff = np.zeros(nstep)
time = np.zeros(nstep)
for i in range(nstep):
    t = i*dt
    time[i] = t
    ff[i] = k_t(a,b,c,time[i])

plt.plot(time,ff)
plt.show()
nstep

In [ ]:
# test of spatial convolution first
a = 20/16.5
b = 0.3
c = 0.008


t0 = 0.0

dt = 1.0
speed = 0.5
width = 1
x_ini=xmin+width

u_sc = np.zeros(xnpts)
y    = np.zeros(xnpts)
for i in range(xnpts):
    ssum = 0.0
    for j in range(xnpts):
        xp = x[j]
        ssum = ssum +  s_d(xp,t0,width,speed,x_ini)*k_x(x[i]-xp) #dark bar
    u_sc[i] = ssum
    y[i] = s(x[i],t0,width,speed,x_ini)    

In [ ]:
f,a = plt.subplots()
a.plot(x,u_sc,label='Response')
a.plot(x,y,label='Stimulation')
#a.set_xlim([-2.5,2.5])
a.legend(frameon=False)
a.set_ylabel('Activity')
a.set_xlabel('Spatial Position')
plt.show()

In [ ]:
# test of temporal convolution
a = 20/16.5
b = 0.3
c = 0.008

x_ini = xmin # starts from the left margin
x0 = -5.     # observartion point

dt = 1.0
speed = 0.1
width = 1


u_tt = np.zeros(nstep)
f = np.zeros(nstep)
time = np.zeros(nstep)

for i in range(nstep):
    t = i*dt
    time[i] = t
    ssum = 0.0
    for j in range(i):
        tp = time[j]
        ssum = ssum + s_d(x0,tp,width,speed,x_ini)*k_t(a,b,c,time[i]-tp) # dt is not included because it is a constant
    u_tt[i] = ssum
    y[i] = s(x[i],t0,width,speed,x0)

In [ ]:
f,a = plt.subplots()
a.plot(time,u_tt,label='Response')
#a.plot(x,y,label='Stimulation')
#a.set_xlim([-2.5,2.5])
a.legend(frameon=False)
a.set_ylabel('Activity')
a.set_xlabel('time (ms)')
plt.show()

## Convolution

$ u(x_0,t_0) = g(v)\int_\infty^{-\infty}dx\int_{-\infty}^{t_0} S(x',t')k(x_0-x',t_0-t')dt'$


$v(t) = \int_{-\infty}^t dt' u(t')B e^{{-\frac{t-t'}{\tau}}}$

In [ ]:
def lowpass(u,npts,t0,B,tau):
    v= np.zeros(npts)
    for ii in range(npts):
        ssum = 0.0
        t=int(t0)
        for j in range(t):
            tp = j
            ssum = ssum + u[ii,j]*np.exp(-(t-tp)/tau)*B
        v[ii] = ssum   
    return v


#=========================================================
# gain control

def gain(vv):
    val = 1
    if vv>0:
        val = 1/(1+vv**4)
    return val

#=========================================================
def F(x,theta):
    if x < theta:
        return 0
    else:
        return x

In [ ]:
# convolution with a moving bar: spatial temporal convolution with feedback




a = 20/16.5
b = 0.3
c = 0.008

dt = 1.0
speed = 0.1
width = 1.0

x_ini = xmin+width # starts from the left margin

xrange = xmax-xmin
x_offset = -xrange/2

tau= 10
B = 0.2

time = np.zeros(nstep)
g = np.zeros(xnpts)
r = np.zeros(xnpts)
pf = np.zeros(xnpts)

for i in range(xnpts): # initialize all the gain = 1 for every point in x
    g[i] = 1.0

u   =np.zeros((xnpts,nstep)) # convoluted results for every step
ss_d=np.zeros((xnpts,nstep))

for j in range(xnpts):
    for i in range(nstep):
        ss_d[j,i]=s_d(x[j],i,width,speed,x_ini)


for i in range(nstep):
    print(i)
    time[i] = i*dt
    t0= time[i]
    print(i)
    #compute spatial-temporal convolution at t0
    for k in range(xnpts):
        x0 = x[k]
        ssum=0.0 # for every obervation point (x0,t0) compute the convolution
        # compute only the needed range for xp and tp from x0 and t0; most of k_t and k_x are zero
        
        xp_min = x0 - 25
        xp_max = x0 + 25
        
        if xp_min < 0 :
            xp_min = xmin
        if xp_max > xmax :
            xp_max = xmax
            
        index_xp_min = int(np.round((xp_min-x_offset)/xrange)*xnpts)
        index_xp_max = int(np.round((xp_max-x_offset)/xrange)*xnpts)
        
        tp_min = t0 - 30
        if tp_min < 0:
            tp_min = 0
        index_tp_min = int(np.round(tp_min/dt))
            
        for l in range(index_xp_min,index_xp_max,1):
            xp = x[l]
            for j in range(index_tp_min,i,1): # integrate only to t0 = time[i]
                tp = time[j]
                ssum = ssum +  ss_d[l,j]*ff[i-j]*gg[k-l]
                # Dark bar
                # dt and dx are not included because they are constant
        u[k,i]= g[k]*ssum
        
    v = lowpass(u,xnpts,t0,B,tau)  # lowpass filtering to get v from u for every point in x
    for ii in range(xnpts):
        g[ii] = gain(v[ii])         # update the gain for next step
        r[ii] = u[ii,i]
        pf[ii]= (1.0-s(x[ii],t0,width,speed,x_ini))*2.0
    plt.plot(g,'r')
    plt.plot(r,'g')
    plt.plot(pf,'b')
    plt.plot(v,'y')
    plt.show()

In [ ]:
dt

In [ ]:
j=10
func = np.zeros(xnpts)
for i in range(xnpts):
    func[i] = u[i,j]
plt.plot(func)

plt.show()

In [ ]:
t0=199
pf = np.zeros(xnpts)
r_out = np.zeros(xnpts)
for i in range(xnpts):
    pf[i]= s(x[i],t0,width,speed,x_ini)
    r_out[i] = u[i,t0]
plt.plot(g,'r')
plt.plot(r_out,'b')
plt.plot(pf,'g')
plt.show()